# Nuvem de partículas (PSO)

```
1. P = população inicial de tamanho m
2. Enquanto critério de parada == false
    a. Para cada partícula p em P
        - Se f(p) > pbest, então pbest = f(p)
    b. gbest = melhor pbest da vizinhaça
    c. Para cada partícula p em P
        - vi = wvi 
                + c1 * r1 (pbest - xi) 
                + c2 * r2 (gbest - xi)
        - pi = pi + vi
3. retorna o gbest
```

In [3]:
from random import uniform
from typing import List
from typing import Dict

from dataclasses import dataclass
from dataclasses import field

@dataclass
class Particula:
    # Tamanho da partícula
    n: int

    # Solução
    x: List[float] = field(default_factory=list)
    
    # Velocidade e deslocamento da partículoa
    v: List[float] = field(default_factory=list)

    # Melhor solução já encontrada pela partícula
    pbest: Dict = field(default_factory=dict)
    
    tipo: str = 'MIN'

    def __post_init__(self):
        if not self.x:
            self.x = [uniform(-10, 10) for _ in range(self.n)]
            self.v = [uniform(-1, 1) for _ in range(self.n)]
       
        self.pbest['x'] = self.x
        self.pbest['fitness'] = 999_999 if self.tipo == 'MIN' else -999_999



In [4]:
# Constantes

# Coeficiente de inércia: valor entre 0,4 e 0,9
W = 0.5 

# Valores empiricos: c1 + c2 = 4
C1 = 2
C2 = 2

# Tamanho da população
M = 10

MAX_ITER = 20

In [15]:
from functools import lru_cache

# Definição do problema 
# @lru_cache(maxsize=200)
def f(p: Particula):
    x1 = p.x[0]
    x2 = p.x[1]

    if (x1 - x2) <= 1: return 999_999

    return abs(2 * x1 ** 2 - 6 * x1 - 9) + \
           abs(2 * x2 ** 2 - 6 * x2 - 9) 

In [16]:
# TODO: Incluir a impressão passo a passo

from copy import deepcopy
from random import random

# 1. P = população inicial de tamanho m
P = [Particula(n=2) for _ in range(M)]

# 2. Enquanto critério de parada == false
g = 0
gbest = deepcopy(P[0])

while g <= MAX_ITER:

    # a. Para cada partícula p em P
    for p in P:

        # - Se f(p) < pbest, então pbest = f(p)
        if p.tipo == 'MIN':
            if f(p) < p.pbest['fitness']:
                p.pbest['fitness'] = f(p)
                p.pbest['x'] = p.x.copy()
        else:
            if f(p) > p.pbest['fitness']:
                p.pbest['fitness'] = f(p)
                p.pbest['x'] = p.x.copy()

    # b. gbest = melhor pbest da vizinhaça
    for p in P:
        if p.tipo == 'MIN':
            if p.pbest['fitness'] < gbest.pbest['fitness']:
                gbest = deepcopy(p)
        else:
            if p.pbest['fitness'] > gbest.pbest['fitness']:
                gbest = deepcopy(p)

    # c. Para cada partícula p em P
    for p in P:

        for i in range(p.n):

            # - vi = wvi 
            #          + c1 * r1 (pbest - xi) 
            #          + c2 * r2 (gbest - xi)
            p.v[i] = W * p.v[i] \
                        + C1 * random() * (p.pbest['x'][i] - p.x[i]) \
                        + C2 * random() * (gbest.x[i] - p.x[i])
      
            # - pi = pi + vi
            p.x[i] += p.v[i]

    g += 1

# 3. retorna o gbest
print(gbest)

Particula(n=2, x=[4.073249440739792, -1.050156839530468], v=[-0.41554716576980405, -0.0017901190827231146], pbest={'x': [4.073249440739792, -1.050156839530468], 'fitness': 0.7501748190566477}, tipo='MIN')
